In [81]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [82]:
data = pd.read_csv('2020-XTern-DS.csv')
#Noticed some faulty lines in Average_Cost, so the values are replaced with the median
data.at[822,'Average_Cost'] = '$10.00'
data.at[1296,'Average_Cost'] = '$10.00'

In [83]:
data.head()

,Restaurant,Latitude,Longitude,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Cook_Time
0,ID_6321,39.262605,-85.837372,"Fast Food, Rolls, Burger, Salad, Wraps",$20.00,$50.00,3.5,12,4,30 minutes
1,ID_2882,39.775933,-85.740581,"Ice Cream, Desserts",$10.00,$50.00,3.5,11,4,30 minutes
2,ID_1595,39.253436,-85.123779,"Italian, Street Food, Fast Food",$15.00,$50.00,3.6,99,30,65 minutes
3,ID_5929,39.029841,-85.332050,"Mughlai, North Indian, Chinese",$25.00,$99.00,3.7,176,95,30 minutes
4,ID_6123,39.882284,-85.517407,"Cafe, Beverages",$20.00,$99.00,3.2,521,235,65 minutes


In [84]:
## First a little pre-processing
def getVal(time):
    return int(time.split(' minutes')[0])
# Convert the Cook Time to a int value
data['Cook_Time'] = data['Cook_Time'].apply(getVal)
# Get a dataframe of a categorical version of the Cusines for predictions
# add a few columns to correspond to the Cuisines
cuisines = data['Cuisines'].unique()
cuisines = ','.join(cuisines)
cuisines = cuisines.split(',')
cuisines = np.unique(cuisines)
Cuisines = pd.DataFrame()
for i in cuisines:
    Cuisines[i] = data['Cuisines'].str.contains(i)
Cuisines = Cuisines.astype(int)
#Turn the prices into values
def getCost(cost):
    return float(cost.split('$')[1])
data["Average_Cost"] = data['Average_Cost'].apply(getCost)
data["Minimum_Order"] = data['Minimum_Order'].apply(getCost)

In [85]:
data

,Restaurant,Latitude,Longitude,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Cook_Time
0,ID_6321,39.262605,-85.837372,"Fast Food, Rolls, Burger, Salad, Wraps",20.0,50.0,3.5,12,4,30
1,ID_2882,39.775933,-85.740581,"Ice Cream, Desserts",10.0,50.0,3.5,11,4,30
2,ID_1595,39.253436,-85.123779,"Italian, Street Food, Fast Food",15.0,50.0,3.6,99,30,65
3,ID_5929,39.029841,-85.332050,"Mughlai, North Indian, Chinese",25.0,99.0,3.7,176,95,30
4,ID_6123,39.882284,-85.517407,"Cafe, Beverages",20.0,99.0,3.2,521,235,65
...,...,...,...,...,...,...,...,...,...,...
2014,ID_4366,39.624978,-85.189212,"North Indian, Pizza, Burger, Continental",40.0,0.0,-,-,-,30
2015,ID_319,39.133719,-85.503020,"Awadhi, Bihari",40.0,99.0,4.1,118,91,30
2016,ID_3104,39.662987,-85.950894,"Fast Food, Beverages",10.0,50.0,NEW,-,-,30
2017,ID_4396,39.301765,-85.662768,"Kebab, Fast Food",10.0,50.0,-,-,-,30


In [86]:
Cuisines.head()

,Afghan,American,Andhra,Arabian,Asian,Assamese,Awadhi,BBQ,Bakery,Bangladeshi,...,Salad,Sandwich,Seafood,South Indian,Street Food,Tamil,Tea,Thai,Tibetan,Wraps
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
Cuisines['AverageCost'] = data['Average_Cost']

In [108]:
Cuisines.corr()['AverageCost'].sort_values()
# What we can see from this list is that people, on average pay the most for Italian food as it is positively correlated with the cost
# We can also see the least expensive food is typically the Street food and Fast food as they are the most inversly correlated which makes sense.

Fast Food      -0.137344
Street Food    -0.126298
South Indian   -0.094809
Mithai         -0.085784
Ice Cream      -0.065949
                  ...   
Finger Food     0.263629
 Italian        0.289543
Continental     0.294754
Italian         0.310286
AverageCost     1.000000
Name: AverageCost, Length: 148, dtype: float64

In [109]:
Cuisines['MinimumOrder'] = data['Minimum_Order']

In [110]:
Cuisines.corr()['MinimumOrder'].sort_values()

Beverages      -0.060499
Desserts       -0.055494
 Beverages     -0.053098
Fast Food      -0.047364
 Desserts      -0.045814
                  ...   
Mughlai         0.128467
Kebab           0.198620
AverageCost     0.204028
 Kebab          0.206150
MinimumOrder    1.000000
Name: MinimumOrder, Length: 149, dtype: float64

In [ ]:
# What we can tell from here is that beverage restraunts have the least minimum order as expected.
# We can also tell that kebab places have the most expensive minimum order based on the correlation with the mimimum order.